<p float="left">
  <img src=nanoHUB_logo_color.png width="500px" height='200px' align="center" /> 
</p>


# Querying Materials for Optical Applications


Shivam Tripathi, Saswat Mishra,  and Alejandro Strachan <br>
 
School of Materials Engineering and Birck Nanotechnology Center, Purdue University, West Lafayette, Indiana 47907, USA <br>

## Overview

The objective of this tool is to find suitable materials for optical applications. We obtain data from [Materials Project](https://materialsproject.org/open) (MP), which uses electronic structure calculations to determine materials' properties. First, we query all the cubic materials in the MP database, then a set of mechanical and optical properties based filters are applied to downselect the materials appropriate for optical applications. We extend the work in Ref. [1] to all the cubic materials and an additional filter based on elastic isotropy is also applied. We predict infrared (IR) and ultraviolet (UV) wavelengths' cutoff from phonon density of states (ph-DOS) and bandgap, respectively.

Notes: 
* This notebook uses tools from [Materials Project](https://materialsproject.org/open) and requires an account with an API key. The users can obtain their API key from https://materialsproject.org/open.
* Run cells in sequential order. In many cases, we will drop columns or rows from pandas DataFrames, and re-running the cell will error out. To re-run notebook begin with cells below the initial query.

## References

1. Kim, Hyunjun, et al. "IR transmission prediction, processing, and characterization of dense La2Ce2O7." Journal of the American Ceramic Society 104.11 (2021): 5659-5670.
2. Batouche, M., et al. "Structural, electronic, optical and elastic properties of XLa2S4 (X= Ba; Ca): Ab initio study." Physica B: Condensed Matter 558 (2019): 91-99.
3. Morales-García, Ángel, Rosendo Valero, and Francesc Illas. "An empirical, yet practical way to predict the band gap in solids by using density functional band structure calculations." The Journal of Physical Chemistry C 121.34 (2017): 18862-18866.


# Loading your API Key
To access data from MP database, users need to enter their own API key in a textbox after running the first cell. Remember to hit "Enter". 

### Load Materials Project API Key

In [ ]:
import os, stat
from IPython.display import clear_output

try:
    user = str(input())
    clear_output()
    if not user.isalnum():
        raise TypeError('Wrong Key')
    if user == None:
        raise TypeError('Empty')
    with open(os.path.expanduser('~/.mpkey.txt'), 'w') as keyfile:
        keyfile.write(user)
    os.chmod(os.path.expanduser('~/.mpkey.txt'), stat.S_IREAD | stat.S_IWRITE)
    del user
    print("Success")
except:
    print("Something seems wrong with your key")

## Libraries

This notebook requires several libraries to be imported. They are separated in blocks depending on their usage.

In [ ]:
# These lines import both libraries and then define an array with elements to be used below

#Pymatgen and sub-libraries
import pymatgen
from pymatgen import MPRester
from pymatgen.core import composition, Element, Structure
from pymatgen.io.vasp import Vasprun
from pymatgen.entries.computed_entries import ComputedEntry
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
from pymatgen.util.plotting import *
from pymatgen.analysis.phase_diagram import *
from pymatgen.apps.borg.hive import VaspToComputedEntryDrone
from pymatgen.entries.compatibility import MaterialsProjectCompatibility
import json

#Misc. Tools
import matplotlib.pyplot as plt
import random
import os
from itertools import combinations
import pandas as pd

#Import API Key
file = open(os.path.expanduser('~/.mpkey.txt'),"r+")
apikey = file.readline()
file.close()
rester = MPRester(apikey)

# Query all cubic materials in MP database

In [ ]:
#Query every cubic structure in the MP database. 
#Grab set of properties from database.
data = rester.query({ "crystal_system": "cubic"},
                    ["task_id","pretty_formula","formula","volume",
                     "density","elements","e_above_hull","elasticity",
                     "spacegroup.number","unit_cell_formula","band_gap","crystal_system","diel"])

First let's check how large our queried database is. Print out the length of the entire database to find the total number of cubic structures that exist in MP database

In [ ]:
print('Number of cubic structures available on the MP database: %s' % len(data))
print('Example output: %s' % data[1])

In [ ]:
df_cubic = pd.DataFrame.from_dict(data) #converting dictionary to panda DataFrame
display(df_cubic)

In [ ]:
#ZnS is currently the most attractive material for IR applications. We are storing the properties of ZnS seperately for comparison.

df_ZnS = df_cubic[df_cubic['pretty_formula'] == "ZnS"] 
df_ZnS = df_ZnS.reset_index(drop=True)
display(df_ZnS)

## Filtering Dataset
### Step 1: Energy stability and bandgap
First, we will remove materials that have energy above the convex hull greater than 50 meV. 0 meV indicates the most stable phase. We will also remove materials which have band gap lesser than 0.5 eV. These filters can be changed below.<br>

In [ ]:
energy_cutoff_value = 50 #maximum allowed value of energy above convex hull [meV]
band_gap_cutoff_value = 0.5 #minmum allowed value of bandgap [eV]
df_cubic = df_cubic[df_cubic['e_above_hull'] <= (energy_cutoff_value/1000)]   
df_cubic = df_cubic[df_cubic['band_gap'] >= (band_gap_cutoff_value)]
df_cubic = df_cubic.reset_index(drop=True)

In [ ]:
print(len(df_cubic)) #Materials with cubic structure, e_above_hull < 0.005 eV, and band_gap > 0.5 eV
df_cubic.to_csv('Query-fs1.csv') #saving data in CSV (Query-filtered-step-1.csv)

### Step 2:  Removing materials without elasticity information

In [ ]:
df_cubic = df_cubic.dropna(subset=['elasticity'])
df_cubic = df_cubic.reset_index(drop=True)

In [ ]:
print(len(df_cubic)) #Materials with cubic structure, e_above_hull < 0.05, band_gap > 0.5 and elasticity information

In [ ]:
#Renaming the properties to improve the readability

df_cubic = df_cubic.rename(columns={"density": "Density", "e_above_hull": "E_Above_Hull",
                   "elasticity": "Elasticity","elements":"Elements",
                   "formula":"formula_dict","pretty_formula":"Formula",
                   "volume":"Volume", "unit_cell_formula":"Unit_Cell_Formula", "band_gap":"Bandgap", "crystal_system":"Structure", "diel":"Dielectric_Properties"})

df_ZnS = df_ZnS.rename(columns={"density": "Density", "e_above_hull": "E_Above_Hull",
                   "elasticity": "Elasticity","elements":"Elements",
                   "formula":"formula_dict","pretty_formula":"Formula",
                   "volume":"Volume", "unit_cell_formula":"Unit_Cell_Formula", "band_gap":"Bandgap", "crystal_system":"Structure", "diel":"Dielectric_Properties"})

display(df_cubic)

In [ ]:
#Extracting the Shear Modulus, Bulk modulus, Elastic anisotropy and Poisson's ration from the Elasticity dictionary and putting in separate columns in panda DataFrame

elastics = df_cubic.Elasticity
G_VRH = []
K_VRH = []
Elastic_Anisotropy = []
poisson_ratio = []

for k in elastics.index: #len(df_cubic))
    G_VRH.append(elastics[k].get('G_VRH')) #Shear Modulus
    K_VRH.append(elastics[k].get('K_VRH')) #Bulk Modulus

    Elastic_Anisotropy.append(elastics[k].get('elastic_anisotropy')) #Elastic Anisotropy
    poisson_ratio.append(elastics[k].get('poisson_ratio'))           #Poisson Ratio
    
elasticity_data = np.column_stack((G_VRH, K_VRH, Elastic_Anisotropy, poisson_ratio))

df_elasticity = pd.DataFrame(elasticity_data,columns=["G_VRH", "K_VRH", "Elastic_Anisotropy", "Poisson_Ratio"])



df_cubic = df_cubic.reset_index(drop=True)
df_elasticity = df_elasticity.reset_index(drop=True)
df_cubic = pd.concat([df_cubic,df_elasticity],axis=1)
display(df_cubic)
df_cubic.to_csv('Query-fs2.csv')

In [ ]:
#Extracting same elastic properties for ZnS and putting in separate columns in panda DataFrame
elastics = df_ZnS.Elasticity
G_VRH = df_ZnS.Elasticity[0].get('G_VRH')
K_VRH = df_ZnS.Elasticity[0].get('K_VRH')
Elastic_Anisotropy = df_ZnS.Elasticity[0].get('elastic_anisotropy')
poisson_ratio = df_ZnS.Elasticity[0].get('poisson_ratio')

    
elasticity_data = np.column_stack((G_VRH, K_VRH, Elastic_Anisotropy, poisson_ratio))

df_elasticity = pd.DataFrame(elasticity_data,columns=["G_VRH", "K_VRH", "Elastic_Anisotropy", "Poisson_Ratio"])


df_ZnS = df_ZnS.reset_index(drop=True)
df_elasticity = df_elasticity.reset_index(drop=True)
df_ZnS = pd.concat([df_ZnS,df_elasticity],axis=1)
display(df_ZnS)

### Step 3:  Bulk modulus
We will remove materials that have bulk modulus lesser than 50 GPa. This filter can be changed below.<br>

In [ ]:
K_VRH_cutoff_value = 50
df_cubic = df_cubic[df_cubic['K_VRH'] >= (K_VRH_cutoff_value)]  
df_cubic = df_cubic.reset_index(drop=True)

In [ ]:
print(len(df_cubic))
df_cubic.to_csv('Query-fs3.csv') 

## Visualization of dataset

In [ ]:
# Create dummy columns to help with plotting select colors

color_new = ['black']*len(df_cubic)

opacity_new = [1]*len(df_cubic)

df_cubic['Color'] = color_new
df_cubic['Opacity'] = opacity_new = [1]*len(df_cubic)

df_ZnS['Color'] = ['black']*len(df_ZnS)
df_ZnS['Opacity'] = [1]*len(df_ZnS)

In [ ]:
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot
import plotly.express as px

plotly.offline.init_notebook_mode(connected=True)

#Layout design. Can change title, font, x/y-axis etc. 
#Image can be directly saved by hovering over image and clicking camera icon. 
layout_BM = go.Layout( hovermode= 'closest',
                   font = dict(family='Times New Roman',size=35),
                   xaxis= dict(title= "Bandgap (eV)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   yaxis= dict(title=  "Bulk Modulus (GPa)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   height = 600,
                   width = 700,
                   
     showlegend= True, legend_font = dict(family='Times New Roman',size=20)                
)

#Scatter plot of collected data. 
trace_all = go.Scatter(x = df_cubic[df_cubic.Color == 'black']['Bandgap'], y = df_cubic[df_cubic.Color == 'black']['K_VRH'], mode = 'markers',
                    marker=dict(size=22, color='black', opacity = 1), text = df_cubic[df_cubic.Color == 'black']['Formula'], name = 'All Queries')

trace_ZnS = go.Scatter(x = df_ZnS[(df_ZnS.Formula == 'ZnS')].loc[:,'Bandgap'],
                         y = df_ZnS[(df_ZnS.Formula == 'ZnS')].loc[:,'K_VRH'],
                         mode = 'markers', marker=dict(size=22, color='red', symbol = 'circle-x', line=dict(width = 2, color = 'black')), text = 'ZnS', name = 'ZnS')

trace_CaLa2S4 = go.Scatter(x = [2.8],
                         y = [58.42],
                         mode = 'markers', marker=dict(size=22, color='yellow', symbol = 'circle-x', line=dict(width = 2, color = 'black')), text = 'CaLa<sub>2</sub>S<sub>4</sub>', name = 'CaLa<sub>2</sub>S<sub>4</sub>')

#CaLa2S4 properties are taken from Ref. [2]


data_BM = [trace_all,trace_ZnS,trace_CaLa2S4]
fig_BM = go.Figure(data_BM, layout=layout_BM)
fig_BM.update_xaxes(automargin=True)


iplot(fig_BM)


### Step 4: Elastic anisotropy
We will remove materials that have elastic anisotropy greater than 1. This filter can be changed below. <br>

In [ ]:
#Elastic anisotropy coefficient represents anisotropy in the material and calculated by lower and upper bounds of shear and bulk modulus for polycrystalling material.
# n = 5(G_max/G_min) + (K_max/kmin) - 6
Elastic_anisotropy_cutoff_value = 1
df_cubic = df_cubic[df_cubic['Elastic_Anisotropy'] <= (Elastic_anisotropy_cutoff_value)] 
df_cubic = df_cubic.reset_index(drop=True)

In [ ]:
print(len(df_cubic))
df_cubic.to_csv('Query-fs4.csv') 

## Visualization of remaining dataset 

In [ ]:
import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

plotly.offline.init_notebook_mode(connected=True)

#Layout design. Can change title, font, x/y-axis etc. 
#Image can be directly saved by hovering over image and clicking camera icon. 
layout_BM_EA = go.Layout( hovermode= 'closest',
                   font = dict(family='Times New Roman',size=35),
                   xaxis= dict(title= "Bandgap (eV)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   yaxis= dict(title=  "Bulk Modulus (GPa)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   height = 600,
                   width = 700,
                   
     showlegend= True, legend_font = dict(family='Times New Roman',size=20)                
)

#Scatter plot of collected data. 
trace_all = go.Scatter(x = df_cubic[df_cubic.Color == 'black']['Bandgap'], y = df_cubic[df_cubic.Color == 'black']['K_VRH'], mode = 'markers',
                    marker=dict(size=22, color='black', opacity = 1), text = df_cubic[df_cubic.Color == 'black']['Formula'], name = 'All Queries')



data_BM_EA = [trace_all,trace_ZnS,trace_CaLa2S4]
fig_BM_EA = go.Figure(data_BM_EA, layout=layout_BM_EA)
fig_BM_EA.update_xaxes(automargin=True)

iplot(fig_BM_EA)

## Query for phonon density of states for IR cutoff calculation

In [ ]:
#IR cutoff calculation based on maximum frequency with ph-DOS between 0 and 0.1
from pymatgen.phonon.dos import PhononDos
df_cubic["IR_cutoff"] = np.nan
for index, row in df_cubic.iterrows():
    m_id = row['task_id']
    try:
        data_DOS = rester.get_phonon_dos_by_material_id(m_id).as_dict()
    except:
        data_DOS = {}
    if data_DOS:
        for i in range(len(data_DOS['frequencies'])-1,0,-1): 

            if(data_DOS['densities'][i]>0.1): 
                max_freq = data_DOS['frequencies'][i+1] #In THz units
                IR_cutoff = (2.998*(10**(14)))/(2.5*max_freq*(10**(12)))  # IR_cutoff (micron) = c(micron/s)/(2.5*max_freq (/s))
                df_cubic.at[index,"IR_cutoff"] = IR_cutoff
                break  
display(df_cubic)

In [ ]:
#IR cutoff calculation for ZnS
df_ZnS["IR_cutoff"] = np.nan
m_id = df_ZnS.at[0,'task_id']
data_DOS_ZnS = rester.get_phonon_dos_by_material_id(m_id).as_dict()
for i in range(len(data_DOS_ZnS['frequencies'])-1,0,-1):    
    if(data_DOS_ZnS['densities'][i]>0.1): 
        max_freq_ZnS = data_DOS_ZnS['frequencies'][i+1] #In THz units  
        IR_cutoff = (2.998*(10**(14)))/(2.5*max_freq_ZnS*(10**(12)))  # IR_cutoff (micron) = c(micron/s)/(2.5*max_freq (/s))
        df_ZnS.at[0,"IR_cutoff"] = IR_cutoff
        break  
display(df_ZnS)

In [ ]:
# Sample ph-DOS plot for visualization of maximum frequency used for IR cutoff calculation

import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

plotly.offline.init_notebook_mode(connected=True)
 
#Image can be directly saved by hovering over image and clicking camera icon. 
layout_DOS_ZnS = go.Layout( hovermode= 'closest',
                   font = dict(family='Times New Roman',size=35),
                   xaxis= dict(title= "Frequency (THz)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   yaxis= dict(title=  "Density of states",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   height = 600,
                   width = 700,
                   
     showlegend= True, legend_font = dict(family='Times New Roman',size=20)                
)
 
DOS_ZnS = go.Scatter(x = data_DOS_ZnS['frequencies'],y = data_DOS_ZnS['densities'], mode = 'lines',text = 'ZnS', name = 'ZnS')
DOS_max_frequency = go.Scatter(x = [max_freq_ZnS] ,y = [0.1], mode = 'markers',marker=dict(size=10, color='black', opacity = 1),text = '&#969;<sub>max</sub>', name = '&#969;<sub>max</sub>')
                            


plot_data_DOS_ZnS = [DOS_ZnS,DOS_max_frequency]

fig_DOS_ZnS = go.Figure(plot_data_DOS_ZnS, layout=layout_DOS_ZnS)

fig_DOS_ZnS = fig_DOS_ZnS.add_shape(type='line', x0=0, y0=0.1, x1=1,y1=0.1,line=dict(color='black',dash="dash"),xref='paper',yref='y')


fig_DOS_ZnS.update_xaxes(automargin=True)

iplot(fig_DOS_ZnS)

### Step 5: Removing materials without IR cutoff information

In [ ]:
df_cubic = df_cubic.dropna(subset=['IR_cutoff'])
df_cubic = df_cubic.reset_index(drop=True)
print(len(df_cubic))

## Visualization of remaining dataset

In [ ]:
#IR cutoff vs bandgap plot

import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

plotly.offline.init_notebook_mode(connected=True)

#Layout design. Can change title, font, x/y-axis etc. 
#Image can be directly saved by hovering over image and clicking camera icon. 
layout_IR = go.Layout( hovermode= 'closest',
                   font = dict(family='Times New Roman',size=40),
                   xaxis= dict(title= "Bandgap (eV)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   yaxis= dict(title=  "IR Cutoff (&#181;m)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   height = 600,
                   width = 700,
                   
     showlegend= False                
)

#Scatter plot of collected data. 

trace_all = go.Scatter(x = df_cubic[df_cubic.Color == 'black']['Bandgap'], y = df_cubic[df_cubic.Color == 'black']['IR_cutoff'], mode = 'markers',
                    marker=dict(size=22, color=df_cubic['K_VRH'], opacity = 1,
                                colorbar=dict(thickness=20),colorbar_title='Bulk Modulus',colorbar_titleside='right',colorbar_title_font = dict(family='Times New Roman',size=40)), text = df_cubic[df_cubic.Color == 'black']['Formula'], name = 'All Queries')

trace_ZnS = go.Scatter(x = df_ZnS[(df_ZnS.Formula == 'ZnS')]['Bandgap'],
                         y = df_ZnS[(df_ZnS.Formula == 'ZnS')]['IR_cutoff'],
                         mode = 'markers', marker=dict(size=22, color='red', symbol = 'circle-x-open', line=dict(width = 3)), text = 'ZnS', name = 'ZnS')


data_IR = [trace_all,trace_ZnS]
fig_IR = go.Figure(data_IR, layout=layout_IR)
fig_IR.update_xaxes(automargin=True)

iplot(fig_IR)

## UV cutoff from bandgap values

In [ ]:
#PBE calculation in DFT underpredict the bandgap, we use correction proposed in Ref. [3] berfore calculating UV cutoff
df_cubic["UV_cutoff"] = np.nan
for index, row in df_cubic.iterrows():
    bandgap_after_correction = (0.998*((1.358*row['Bandgap'])+0.904))+0.014 
    UV_cutoff = 1.2398/bandgap_after_correction    #eV to micron converson
    df_cubic.at[index,"UV_cutoff"] = UV_cutoff
display(df_cubic)
df_cubic.to_csv('Query-fs5.csv') 

In [ ]:
#UV cutoff for ZnS
df_ZnS["UV_cutoff"] = np.nan
bandgap_after_correction = (0.998*((1.358*df_ZnS.at[0,'Bandgap'])+0.904))+0.014
UV_cutoff = 1.2398/bandgap_after_correction
df_ZnS.at[0,"UV_cutoff"] = UV_cutoff
display(df_ZnS)

## Visualization of IR vs UV cutoff for remaining dataset

In [ ]:
#IR cutoff vs UV cutoff plot

import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

plotly.offline.init_notebook_mode(connected=True)

#Layout design. Can change title, font, x/y-axis etc. 
#Image can be directly saved by hovering over image and clicking camera icon. 
layout_IR_UV = go.Layout( hovermode= 'closest',
                   font = dict(family='Times New Roman',size=40),
                   xaxis= dict(title= "UV Cutoff (&#181;m)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   yaxis= dict(title=  "IR Cutoff (&#181;m)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   height = 600,
                   width = 700,
                   
     showlegend= False                
)

#Scatter plot of collected data. 

trace_all = go.Scatter(x = df_cubic[df_cubic.Color == 'black']['UV_cutoff'], y = df_cubic[df_cubic.Color == 'black']['IR_cutoff'], mode = 'markers',
                    marker=dict(size=22, color=df_cubic['K_VRH'], opacity = 1,
                                colorbar=dict(thickness=20),colorbar_title='Bulk Modulus',colorbar_titleside='right',colorbar_title_font = dict(family='Times New Roman',size=40)), text = df_cubic[df_cubic.Color == 'black']['Formula'], name = 'All Queries')

trace_ZnS = go.Scatter(x = df_ZnS[(df_ZnS.Formula == 'ZnS')]['UV_cutoff'],
                         y = df_ZnS[(df_ZnS.Formula == 'ZnS')]['IR_cutoff'],
                         mode = 'markers', marker=dict(size=22, color='red', symbol = 'circle-x-open', line=dict(width = 3)), text = 'ZnS', name = 'ZnS')


data_IR_UV = [trace_all,trace_ZnS]
fig_IR_UV = go.Figure(data_IR_UV, layout=layout_IR_UV)
fig_IR_UV.update_xaxes(automargin=True)

iplot(fig_IR_UV)

### Step 6: Removing materials without dielectric information

In [ ]:
df_cubic = df_cubic.dropna(subset=['Dielectric_Properties'])
df_cubic = df_cubic.reset_index(drop=True)
print(len(df_cubic))

In [ ]:
#Extracting the refractive index, electronic and total dielectric constants from the Dielectric properties dictionary and putting in separate columns in panda Dataframe
dielectrics = df_cubic.Dielectric_Properties
D_electronic = []
D_total = []
Refractive_Index = []


for k in dielectrics.index: #len(df_cubic))
    D_electronic.append(dielectrics[k].get('poly_electronic')) #Electronic 
    D_total.append(dielectrics[k].get('poly_total'))           #Electronic + Ionic
    Refractive_Index.append(dielectrics[k].get('n'))           # Refractive Index 


    

dielectrics_data = np.column_stack((D_electronic, D_total, Refractive_Index))

df_dielectrics = pd.DataFrame(dielectrics_data,columns=["D_electronic","D_total", "Refractive_Index"])



df_cubic = df_cubic.reset_index(drop=True)
df_dielectrics = df_dielectrics.reset_index(drop=True)
df_cubic = pd.concat([df_cubic,df_dielectrics],axis=1)
display(df_cubic)
df_cubic.to_csv('Query-fs6.csv') 

In [ ]:
#Extracting same dielectric properties for ZnS and putting in separate columns in panda DataFrame
dielectrics = df_ZnS.Dielectric_Properties
D_electronic = df_ZnS.Dielectric_Properties[0].get('poly_electronic')
D_total = df_ZnS.Dielectric_Properties[0].get('poly_total')
Refractive_Index = df_ZnS.Dielectric_Properties[0].get('n')

dielectrics_data = np.column_stack((D_electronic, D_total, Refractive_Index))

df_dielectrics = pd.DataFrame(dielectrics_data,columns=["D_electronic","D_total", "Refractive_Index"])



df_ZnS = df_ZnS.reset_index(drop=True)
df_dielectrics = df_dielectrics.reset_index(drop=True)
df_ZnS = pd.concat([df_ZnS,df_dielectrics],axis=1)
display(df_ZnS)


## Visualization of remaining dataset

In [ ]:
#Markers are color coded for refractive index

import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

plotly.offline.init_notebook_mode(connected=True)

#Layout design. Can change title, font, x/y-axis etc. 
#Image can be directly saved by hovering over image and clicking camera icon. 
layout_IR_UV_RI = go.Layout( hovermode= 'closest',
                   font = dict(family='Times New Roman',size=35),
                   xaxis= dict(title= "UV Cutoff (&#181;m)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   yaxis= dict(title=  "IR Cutoff (&#181;m)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   height = 600,
                   width = 700,
                   
     showlegend= False                  
)

#Scatter plot of collected data. 

trace_all = go.Scatter(x = df_cubic[df_cubic.Color == 'black']['UV_cutoff'], y = df_cubic[df_cubic.Color == 'black']['IR_cutoff'], mode = 'markers',
                         marker=dict(size=22, color=df_cubic['Refractive_Index'], opacity = 1,
                                     colorbar=dict(thickness=20),colorbar_title='Refractive Index',colorbar_titleside='right',colorbar_title_font = dict(family='Times New Roman',size=40)), text = df_cubic[df_cubic.Color == 'black']['Formula'], name = 'All Queries')

trace_ZnS = go.Scatter(x = df_ZnS[(df_ZnS.Formula == 'ZnS')].loc[:,'UV_cutoff'],
                         y = df_ZnS[(df_ZnS.Formula == 'ZnS')].loc[:,'IR_cutoff'],
                         mode = 'markers', marker=dict(size=22, color='red',  symbol = 'circle-x-open',line=dict(width = 3)), text = 'ZnS', name = 'ZnS')


data_IR_UV_RI = [trace_all,trace_ZnS]
fig_IR_UV_RI = go.Figure(data_IR_UV_RI, layout=layout_IR_UV_RI)
fig_IR_UV_RI.update_xaxes(automargin=True)


iplot(fig_IR_UV_RI)


In [ ]:
#Markers are color coded for total dielectric constant

import plotly
import plotly.graph_objs as go
from plotly.offline import iplot

plotly.offline.init_notebook_mode(connected=True)

#Layout design. Can change title, font, x/y-axis etc. 
#Image can be directly saved by hovering over image and clicking camera icon. 
layout_IR_UV_DC = go.Layout( hovermode= 'closest',
                   font = dict(family='Times New Roman',size=35),
                   xaxis= dict(title= "UV Cutoff (&#181;m)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   yaxis= dict(title=  "IR Cutoff (&#181;m)",zeroline= False, gridwidth= 2,title_font=dict(family='Times New Roman',size=35)),
                   height = 600,
                   width = 700,
                   
     showlegend= False                  
)

#Scatter plot of collected data. 

trace_all = go.Scatter(x = df_cubic[df_cubic.Color == 'black']['UV_cutoff'], y = df_cubic[df_cubic.Color == 'black']['IR_cutoff'], mode = 'markers',
                         marker=dict(size=22, color=df_cubic['D_total'], opacity = 1,
                                     colorbar=dict(thickness=20),colorbar_title='Dielectric Constant',colorbar_titleside='right',colorbar_title_font = dict(family='Times New Roman',size=40)), text = df_cubic[df_cubic.Color == 'black']['Formula'], name = 'All Queries')

trace_ZnS = go.Scatter(x = df_ZnS[(df_ZnS.Formula == 'ZnS')].loc[:,'UV_cutoff'],
                         y = df_ZnS[(df_ZnS.Formula == 'ZnS')].loc[:,'IR_cutoff'],
                         mode = 'markers', marker=dict(size=22, color='red',  symbol = 'circle-x-open',line=dict(width = 3)), text = 'ZnS', name = 'ZnS')


data_IR_UV_DC = [trace_all,trace_ZnS]
fig_IR_UV_DC = go.Figure(data_IR_UV_DC, layout=layout_IR_UV_DC)
fig_IR_UV_DC.update_xaxes(automargin=True)


iplot(fig_IR_UV_DC)

## Download the filetered dataset

In [ ]:
#Download file by clicking on the name (Query-fsi.csv - Query-filtered-step-i.csv)
from IPython.display import FileLink
FileLink('Query-fs1.csv')

In [ ]:
FileLink('Query-fs2.csv')

In [ ]:
FileLink('Query-fs3.csv')

In [ ]:
FileLink('Query-fs4.csv')

In [ ]:
FileLink('Query-fs5.csv')

In [ ]:
FileLink('Query-fs6.csv')